In [148]:
import pandas as pd
import re

In [149]:
data =pd.read_excel('Ozurgeti.xlsx')                   

In [150]:
data.head(4)
data.columns

Index(['საიდენტიფიკაციო ნომერი', 'პირადი ნომერი',
       'ორგანიზაციულ-სამართლებრივი ფორმა', 'სუბიექტის დასახელება',
       'რეგიონი (იურიდიული)', 'მუნიციპალიტეტი/ქალაქი (იურიდიული)',
       'დაბის/თემის/სოფლის საკრებულო (იურიდიული)', 'დაბა/სოფელი (იურიდიული)',
       'იურიდიული მისამართი', 'რეგიონი (ფაქტობრივი)',
       'მუნიციპალიტეტი/ქალაქი (ფაქტობრივი)',
       'დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)', 'დაბა/სოფელი (ფაქტობრივი)',
       'ფაქტობრივი მისამართი', 'საქმიანობის კოდი NACE Rev.2',
       'საქმიანობის დასახელება NACE Rev.2', 'პარტნიორი', 'ხელმძღვანელი',
       'ტელეფონი', 'ელფოსტა', 'საკუთრების ფორმა',
       'აქტიური ეკონომიკური სუბიექტები', 'ბიზნესის ზომა'],
      dtype='object')

In [151]:
data.head(3
          )

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,რეგიონი (იურიდიული),მუნიციპალიტეტი/ქალაქი (იურიდიული),დაბის/თემის/სოფლის საკრებულო (იურიდიული),დაბა/სოფელი (იურიდიული),იურიდიული მისამართი,რეგიონი (ფაქტობრივი),...,ფაქტობრივი მისამართი,საქმიანობის კოდი NACE Rev.2,საქმიანობის დასახელება NACE Rev.2,პარტნიორი,ხელმძღვანელი,ტელეფონი,ელფოსტა,საკუთრების ფორმა,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა
0,100015680,6.200100e+10,იმ,ზურაბ შელეგია,ქ. თბილისი,ნაძალადევის რაიონი,NaN,NaN,ქ. თბილისის ნაძალადევის რაიონში / მანჯგალაძის 29,გურია,...,ოზურგეთის რაიონში/დაბ. ურეკიƒ.“დდ„დ‰დˆ,47.11.0,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",ზურაბ შელეგია - 100,ზურაბ შელეგია,NaN,NaN,კერძო ადგილობრივი საკუთრება,NaN,NaN
1,100019383,6.200100e+10,იმ,მარინე ჭელიძე,ქ. თბილისი,ნაძალადევის რაიონი,NaN,NaN,"ქ. თბილისის ნაძალადევის რაიონში / სანზონა კ.6,ბ.1",გურია,...,NaN,47.11.0,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",მარინე ჭელიძე - 100,მარინე ჭელიძე,NaN,NaN,კერძო ადგილობრივი საკუთრება,NaN,NaN
2,100085300,3.300100e+10,იმ,დავით ანთაძე,გურია,ოზურგეთის მუნიციპალიტეტი,ბახვის თემის საკრებულო,NaN,ბახვის თემის საკრებულო,გურია,...,ბახვის თემის საკრებულო,NaN,NaN,დავით ანთაძე - 100,დავით ანთაძე,NaN,NaN,კერძო ადგილობრივი საკუთრება,NaN,NaN


In [152]:
data = data[['საიდენტიფიკაციო ნომერი',
             'პირადი ნომერი',
             'დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)',
             'ორგანიზაციულ-სამართლებრივი ფორმა',
             'სუბიექტის დასახელება','ფაქტობრივი მისამართი',
             'საქმიანობის დასახელება NACE Rev.2',
             'აქტიური ეკონომიკური სუბიექტები',
              'ბიზნესის ზომა']]

In [153]:
data.head(4)

,საიდენტიფიკაციო ნომერი,პირადი ნომერი,დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი),ორგანიზაციულ-სამართლებრივი ფორმა,სუბიექტის დასახელება,ფაქტობრივი მისამართი,საქმიანობის დასახელება NACE Rev.2,აქტიური ეკონომიკური სუბიექტები,ბიზნესის ზომა
0,100015680,6.200100e+10,NaN,იმ,ზურაბ შელეგია,ოზურგეთის რაიონში/დაბ. ურეკიƒ.“დდ„დ‰დˆ,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",NaN,NaN
1,100019383,6.200100e+10,ურეკის სადაბო საკრებულო,იმ,მარინე ჭელიძე,NaN,"საცალო ვაჭრობა არასპეციალიზებულ მაღაზიებში, უპ...",NaN,NaN
2,100085300,3.300100e+10,ბახვის თემის საკრებულო,იმ,დავით ანთაძე,ბახვის თემის საკრებულო,NaN,NaN,NaN
3,100096389,3.300101e+10,NaN,იმ,სოსო სადრაძე,შემოქმედის,"სხვა ხილის, კენკროვნების და კაკლისებრების მოყვანა",NaN,NaN


In [154]:
data = data[(data['დაბის/თემის/სოფლის საკრებულო (ფაქტობრივი)'].str.contains('ქ. ოზურგეთი', na=False)) ]


In [155]:
data = data[~(data['ფაქტობრივი მისამართი'].isnull())]


In [156]:
def clean_text(arg):
    cleaned = re.sub(r'[^\w\s]',' ',arg)
    return cleaned

In [157]:
data['ნომერი'] = data['ფაქტობრივი მისამართი'].apply(lambda x: "".join([i for i in x if i.isnumeric()]))

In [158]:
data["ქუჩა"] = data["ფაქტობრივი მისამართი"].apply(clean_text)

In [159]:
data['ქუჩა'] = data['ქუჩა'].apply(lambda x: " ".join([re.sub('\d+', '',i)  for i in x ]))

In [160]:
data['ქუჩა'] = data['ქუჩა'].apply(lambda x: " ".join([re.sub(r'\\N', '',i)  for i in x ]))

In [161]:
data['ქუჩა'] =data['ქუჩა'].apply(lambda x: ''.join([i for i in x if i != 'N']))

In [162]:
data['ქუჩა'] = data['ქუჩა'].apply(lambda x: ' '.join([i for i in x.split() if i.isnumeric() or len(i) >2]))

In [163]:
data.to_excel("test.xlsx", index=False)